# gtfParser Class

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#######################################################################################
###                                                                                 ###
###     Copyright (C) 2019  Zhongxu ZHU, CityU, 20200925                            ###
#######################################################################################

# https://github.com/Jverma/GFF-Parser



class gtfParser:
    def __init__(self, input_file):
        import sys
        self.data = {}
        self.dict = {}
        self.gene_attributes_dict = {} # ZZX
        self.transcriptID_geneID_dict = {} # ZZX

        sys.stderr.write("#####################\nParsing reference gtf file: " + input_file + '\n#####################\n')

        for line in open(input_file):
            if line.startswith("#"): continue
            record = line.strip().split("\t")
            sequence_name = record[0]
            source = record[1]
            feature = record[2]
            start = int(record[3])
            end = int(record[4])
            if (record[5] != '.'):
                score = record[5]
            else:
                score = None
            strand = record[6]
            if (record[7] != '.'):
                frame = record[7]
            else:
                frame = None
            attributes = record[8].split(';')
            attributes = [x.strip() for x in attributes[0:-1]] # ZZX
            attributes = {x.split(' ')[0]: x.split(' ')[1].strip("\"") for x in attributes if " " in x} # ZZX

            if not (sequence_name in self.data): self.data[sequence_name] = []
            alpha = {'source': source, 'feature': feature, 'start': start, 'end': end, 'score': score, 'strand': strand,
                     'frame': frame}
            # python 3 .items(), python 2 .iteritems() ZZX
            for k, v in attributes.items(): alpha[k] = v
            self.data[sequence_name].append(alpha)

        # ZZX
        for k, v in self.data.items():
            for alpha in v:
                gene_id = alpha["gene_id"]
                transcript_id = alpha["transcript_id"]

                self.transcriptID_geneID_dict[transcript_id] = gene_id
                if gene_id in self.gene_attributes_dict.keys():
                    self.gene_attributes_dict[gene_id].append(alpha)
                else:
                    self.gene_attributes_dict[gene_id] = list()
                    self.gene_attributes_dict[gene_id].append(alpha)


    def getRecordsByID(self, id, attType, attValue):
        """ Gets all the features for a given gene.
            Parameters
            ----------
            id : Identifier of the gene (gene_id) or mRNA (transcript_id).

            Returns
            -------
            A list of dictionaries where each dictionary contains the
            informations about features for the transcript.
            """

        att_list = []
        if id in self.gene_attributes_dict.keys():
            for x in self.gene_attributes_dict[id]:
                if ( attType in x.keys() and  x[attType] == attValue ):
                    att_info = x
                    att_list.append(att_info)
        elif id in self.transcriptID_geneID_dict.keys():
            for x in self.gene_attributes_dict[self.transcriptID_geneID_dict[id]]:
                if ( attType in x.keys() and  x[attType] == attValue and x["transcript_id"] == id):
                    att_info = x
                    att_list.append(att_info)
        else:
            sys.stderr.write("Could not find ID "+id+'\n')
            sys.exit(1)

        return att_list

# Initiating

In [6]:


from optparse import OptionParser
import sys
import fastaparser

"""
Description
"""
__author__    = "Zhongxu"
__copyright__ = "Copyright 2020, Planet Earth"

sys.argv = ["", "-c", "/data/home2/Zhongxu/tmp/ORFFinder/HCC-Organoid/candidate.aa.fa",
                "-r", "/data/home2/Zhongxu/tmp/ORFFinder/HCC-Organoid/ref.aa.fa",
                "-m", "/data/home2/Zhongxu/tmp/ORFFinder/HCC-Organoid/transcript.gene.map",
                "-o", "/data/home2/Zhongxu/tmp/ORFFinder/HCC-Organoid/orf.analysis.tsv",
                "-g", "/data/home2/Zhongxu/Ref/refSeq.hg38.gtf",
                #"-g", "/data/home2/Zhongxu/ref/refSeq.hg19.gtf",
                #"-s", "/data/home2/Zhongxu/work/cuhk-crc/analysis/t.gtf"
                #"-s", "/data/home2/Zhongxu/work/pbFLNC/c99i95refMin2/gff/s2.clean.gtf"
                "-s", "/dataserver145/genomics/zhongxu/work/HCC-organoid-AS/analysis/02gff/02organoid/1prepareMergedGff/all.5.filter.gtf"
            ]

parser = OptionParser()
parser.add_option("-c", "--candidate", dest="candidate",
                  help="candidate protein sequence predicted by ORFFinder", metavar="FILE")
parser.add_option("-r", "--reference", dest="reference",
                  help="reference protein sequence", metavar="FILE")
parser.add_option("-m", "--map", dest="idmap",
                  help="1st col: transcript ID, 2ed col: Gene ID", metavar="FILE")
parser.add_option("-o", "--output", dest="out",
                  help="Output file path", metavar="FILE")
parser.add_option("-g", "--refgtf", dest="refgtf",
                  help="GTF file path", metavar="FILE")
parser.add_option("-s", "--usergtf", dest="usergtf",
                  help="GTF file path", metavar="FILE")
parser.add_option("-b", "--blast", dest="blast",
                  help="Blast file path", metavar="FILE")
#-----------------------------------------------------


def proteinSimilarity(seq1,seq2):
    from Bio import pairwise2 as pw2
    seq_length = min(len(seq1), len(seq2))
    global_align = pw2.align.globalxx(seq1, seq2)
    matches = global_align[0][2]
    percent_match = (matches / seq_length) * 100
    seq2_res = global_align[0][0] + '\n' + global_align[0][1]
    return round( percent_match,1 ), seq2_res

#-----------------------------------------------------
(options, args) = parser.parse_args()
#-----------------------------------------------------

# Parse reference sequence

In [ ]:

#-----------------------------------------------------

# parsing this file to store gene and transcript mapping information
transcriptGeneMap = {line.strip().split("\t")[0]:line.strip().split("\t")[1] for line in open(options.idmap)}
geneTranscriptListMap = {}

for line in open(options.idmap):
    gene = line.strip().split("\t")[1]
    transcript = line.strip().split("\t")[0]
    if not gene in geneTranscriptListMap.keys():
        geneTranscriptListMap[gene]=[]
    geneTranscriptListMap[gene].append(transcript)
        


# store CDS start and end position
refCDSStartEndMap = {}
ref_seq_gtf = gtfParser(options.refgtf)

# parsing ref protein sequence and store cds start and end
refIdSeqMap = {}

with open(options.reference) as fasta_file:
    parser = fastaparser.Reader(fasta_file)
    for seq in parser:
        if seq.sequence_as_string() == "SEQUENCE UNAVAILABLE": continue
        for e in seq.id.split(";"):
            # obtain cds start and end
            refmRNA_start = ref_seq_gtf.getRecordsByID(e, "feature", "start_codon")
            start = ''
            end = ''
                        
            if len(refmRNA_start) != 1:
                start = ''
                end = ''
            else:
                strand = refmRNA_start[0]["strand"]
                if strand == '+':                
                    start = refmRNA_start[0]["start"]
                else: # 如果在负链
                    start = refmRNA_start[0]["end"]
                
                refmRNA_end = ref_seq_gtf.getRecordsByID(e, "feature", "stop_codon")
                if (len(refmRNA_end) != 1):
                    end = ''
                else:
                    if strand == '+':
                        end = refmRNA_end[0]["end"]
                    else:
                        end = refmRNA_end[0]["start"]           
            
            refCDSStartEndMap[e] = str(start)+"|"+str(end)
            # remove * at the end of sequence and remove the first M amino acid
            refIdSeqMap[e] = seq.sequence_as_string()[1:-1]     
del ref_seq_gtf



#-------------------------------------------
# 将blast的结果读到map中，ORFID：line #结果是best的，一个orf只对应一行

# parsing ref protein sequence and store cds start and end
user_seq_gtf = gtfParser(options.usergtf)
userCDSStartEndMap = {}

# parsing predicted ORF and store cds start and end
canIdSeqMap = {}
def mapTranscriptPosToGenomic(gtf, transcriptID, start, end):
    records = gtf.getRecordsByID(transcriptID, "feature", "exon")
    strand = records[0]["strand"]
    
    regions = []
    
    for record in records:
        e_start = record["start"]
        e_end   = record["end"]
        regions.extend(list(range(e_start,e_end+1)))
    
    if strand == '+':
        start = regions[int(start)] # 本来打算要start-1的，不知道为什么不减1得到的是正确的
        end   = regions[int(end)]
    else:
        start = regions[-1*int(start)-1] # 本来打算start，不知道为什么必须减1得到的是正确的
        end   = regions[-1*int(end)-1]
    return start, end


with open(options.candidate) as fasta_file:
    parser = fastaparser.Reader(fasta_file)

    for seq in parser:
        
        id = seq.id.split("_",1)[1].split(":")[0]
        start = seq.id.split(":")[1]
        end = seq.id.split(":")[2].split(" ")[0]
        orf = id+":"+start+":"+end
        
        if id in canIdSeqMap.keys(): # 选择最长的那个
            if ( (len(seq.sequence_as_string()) -1 ) > len(canIdSeqMap[id]) ): # 减1是不考虑第一个M氨基酸
                canIdSeqMap[id] = seq.sequence_as_string()[1:]
                g_start, g_end = mapTranscriptPosToGenomic(user_seq_gtf, id, start, end)
                userCDSStartEndMap[id] = str(g_start)+"|"+str(g_end)
        else:
            canIdSeqMap[id] = seq.sequence_as_string()[1:]
            g_start, g_end = mapTranscriptPosToGenomic(user_seq_gtf, id, start, end)
            userCDSStartEndMap[id] = str(g_start)+"|"+str(g_end)
        
del user_seq_gtf
#-------------------------------------------

__main__:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='/data/home2/Zhongxu/tmp/ORFFinder/HCC-Organoid/transcript.gene.map' mode='r' encoding='UTF-8'>
__main__:7: ResourceWarning: unclosed file <_io.TextIOWrapper name='/data/home2/Zhongxu/tmp/ORFFinder/HCC-Organoid/transcript.gene.map' mode='r' encoding='UTF-8'>
#####################
Parsing reference gtf file: /data/home2/Zhongxu/Ref/refSeq.hg38.gtf
#####################
__main__:22: ResourceWarning: unclosed file <_io.TextIOWrapper name='/data/home2/Zhongxu/Ref/refSeq.hg38.gtf' mode='r' encoding='UTF-8'>


# Parsing blast result

In [19]:
transcriptGeneMap['All-NR_024540']

refCDSStartEndMap['NM_021170']

'999973|999059'

# Analysis

In [20]:
#-------------------------------------------
if options.out:
    ofs = open(options.out,"w", encoding='utf-8')
else:
    ofs = sys.stdout
    
def errInfo(err, id, seq, transcript, refSeq, similarity, seqLength, refSeqLength, 
            len_diff, upstreamSimilarity, u_similar_res, downstreamSimilarity, d_similar_res, stopCodonStateOneRound):
    sys.stderr.write(err)
    sys.stderr.write(id+": "+ seq +"\n")
    sys.stderr.write(transcript+": "+ refSeq +"\n")
    sys.stderr.write("Similarity: "+ str(similarity) +"\n")  
    sys.stderr.write("Candidate length: "+ str(seqLength) +"\n")  
    sys.stderr.write("Reference length: "+ str(refSeqLength) +"\n")  
    sys.stderr.write("Length different: "+ str(len_diff) +"\n")  
    sys.stderr.write("Upstream similarity (10aAA), similarity: "+ str(upstreamSimilarity) +"\n")  
    sys.stderr.write(u_similar_res +"\n")  
    sys.stderr.write("Downstream similarity (20aAA), similarity: "+ str(downstreamSimilarity) +"\n")  
    sys.stderr.write(d_similar_res +"\n")  
    sys.stderr.write("Stop codon status "+stopCodonStateOneRound+'\n')
    sys.stderr.write("Unknown condition, please check ----- \n")
    sys.exit(1)
    
ofs.write("\t".join(["Gene", "Isoform", "CDS Start", "CDS End", "Class", 
                     "Protein Length" ,"Frame", "Start Codon","Stop Codon", "Reference","Protein Length Change","Predicted Protein Sequence"])+'\n')

frameMap = { 'unchanged': 1,
             "5' elongation": 2,
             "3' elongation": 2,
             "insert": 2,
             "delete": 2,
             "contain known orf": 2,
             'in-frame change': 3,
             'frameshift': 4,
             'novel start': 5,
             'novel transcript': 5,
             'novel': 5 ,
             'no reference orf': 5,
             'todo1': 0 ,
             'todo2': 0 ,
             'todo3': 0}
classMap ={ 0: "todo", 1: "unchanged", 2:"elongation", 3:"in-frame change", 4:"frameshift", 5:"novel"}

# impact priority: unchanged < elongation < inframe < frameshift < novel <- ''

for id, seq in canIdSeqMap.items():
    gene = '' #得到基因
    
    if not id in transcriptGeneMap.keys():
        continue # 不在关注的基因内
    
    if ("-N" in id):  # 如果是参考的话，则不考虑
        gene = transcriptGeneMap[id]
        nm = id.replace("All-","")
        if nm.startswith("NR") or not nm in refCDSStartEndMap.keys():
            ofs.write("\t".join([gene, id, '', '', "known", "", "known", "known", "", ""])+"\n")
            continue
        nm_s_e = refCDSStartEndMap[nm]
        nm_s_e = nm_s_e.split("|")
        ofs.write("\t".join([gene, id, nm_s_e[0], nm_s_e[1], "known", str(len(refIdSeqMap[nm])), "known", "known", "", ""])+"\n")
        continue     # 如果是参考的话，则不考虑
    
    user_transcript_start = int(userCDSStartEndMap[id].split("|")[0])
    user_transcript_end   = int(userCDSStartEndMap[id].split("|")[1])

    gene = transcriptGeneMap[id] #得到基因
    
    
    transcript_in_gene = geneTranscriptListMap[gene] # 得到基因内的转录本
    transcript_in_gene = [t for t in transcript_in_gene if not t.startswith("A")] # 只考虑参考转录本，不以ALL-开头
    
    
    frameshiftStateFinal = []       # 单个基因
    frameshiftTranscriptFinal = []  # 单个基因
    stopCodonStateFinal = []       # 单个基因
    lengthChangeFinal = []
    startCodonStateFinal = []
    
    #sys.stderr.write(id,str(user_transcript_start),str(user_transcript_end))
    seqLength    = len(seq) #待研究的转录本信息
    strand = ''
    if user_transcript_start - user_transcript_end > 0:
        strand = '+'
    elif user_transcript_start - user_transcript_end < 0:
        strand = '-'
    maximum_similiary = 0
      
    for transcript in transcript_in_gene: # 遍历基因内的参考转录本
        
        frameshiftStateOneRound = '' # 单个transcript
        stopCodonStateOneRound = ''  # 单个transcript
        startCodonStateOneRound = ''
        
        if not transcript in refIdSeqMap.keys():
            continue # 如果没有氨基酸序列，则continue
        refSeq = refIdSeqMap[transcript]
        
        refSeqLength = len(refSeq) # 参考转录本信息
        similarity, s_similar_res = proteinSimilarity(seq, refSeq)
        upstreamSimilarity, u_similar_res = proteinSimilarity(seq[0:10], refSeq[0:10]) # 前十个氨基酸序列的相似性
        downstreamSimilarity, d_similar_res = proteinSimilarity(seq[-20:], refSeq[-20:]) # 后二十个氨基酸序列的相似性
        if similarity > maximum_similiary:
            maximum_similiary = similarity
        

        pos_split = refCDSStartEndMap[transcript].split("|")
        if pos_split[1] == '': continue
        ref_start = int(pos_split[0])
        ref_end = int(pos_split[1])
              
        # 判断终止密码子的位置 stopCodon
        if user_transcript_end == ref_end:
            stopCodonStateOneRound = "unchanged"
        elif strand == '+':
            if user_transcript_end > ref_end:
                stopCodonStateOneRound = "late"
            else:
                stopCodonStateOneRound = "early"
        elif strand == "-":
            if user_transcript_end > ref_end:
                stopCodonStateOneRound = "early"
            else:
                stopCodonStateOneRound = "late" 
                
        # 判断起始密码子的位置
        if user_transcript_start == ref_start:
            startCodonStateOneRound = "unchanged"
        elif strand == '+':
            if user_transcript_start > ref_start:
                startCodonStateOneRound = "late"
            else:
                startCodonStateOneRound = "early"
        elif strand == "-":
            if user_transcript_start > ref_start:
                startCodonStateOneRound = "early"
            else:
                startCodonStateOneRound = "late"        
        

                
        # 判断移码情况 frameshift
        len_diff = seqLength - refSeqLength   
        if user_transcript_start == ref_start and user_transcript_end == ref_end :
            if seq == refSeq:
                frameshiftStateOneRound = "unchanged"
            elif similarity >= 95 and (upstreamSimilarity!=100 or downstreamSimilarity!=100):
                frameshiftStateOneRound = "in-frame change" # 相似性高的，归为in-frame
            elif downstreamSimilarity >= 90 and (seq[0:5] in refSeq or refSeq[0:5] in seq or upstreamSimilarity >= 90):
                frameshiftStateOneRound = "in-frame change" # 相似性高，且前面相同
            elif downstreamSimilarity <= 50 and (seq[0:5] in refSeq or refSeq[0:5] in seq or upstreamSimilarity >= 90):
                frameshiftStateOneRound = "frameshift"  # 前面一致，后面不一致
            elif downstreamSimilarity >= 50 and seq[-1] == refSeq[-1] and upstreamSimilarity >= 90 and similarity >= 90:
                frameshiftStateOneRound = "in-frame change" # 前面不一致，后面最后一个碱基，且后面有几个碱基一致
            elif downstreamSimilarity >= 90 and (not seq[0:5] in refSeq and not refSeq[0:5] in seq):
                frameshiftStateOneRound = "novel transcript" # 虽然后半部分相同，但前面不同
            elif downstreamSimilarity <= 50 and upstreamSimilarity <= 50 and (similarity <= 50 or (similarity > 80  and abs(len_diff)) > 200 ) :
                frameshiftStateOneRound = "novel transcript" # 前后都不同，且相似性低
            else:
                errInfo("Same start and end codon, but different sequences\n", id, seq, transcript, refSeq, similarity, seqLength, refSeqLength, 
                                    len_diff, upstreamSimilarity, u_similar_res, downstreamSimilarity, d_similar_res, stopCodonStateOneRound)
        elif (seq[0:5] in refSeq or refSeq[0:5] in seq) and (seq[-5:] in refSeq or refSeq[-5:] in seq):
            frameshiftStateOneRound = "in-frame change"
        elif user_transcript_start == ref_start and user_transcript_end != ref_end:
            if stopCodonStateOneRound == "late":
                if refSeq[-5:] in seq or upstreamSimilarity >= 90:
                    frameshiftStateOneRound = "in-frame change"
                else:
                    frameshiftStateOneRound = "frameshift"
            elif stopCodonStateOneRound == "early":
                if seq[-5:] in refSeq or upstreamSimilarity >= 90:
                    frameshiftStateOneRound = "in-frame change"
                else:
                    frameshiftStateOneRound = "frameshift"
            else:
                errInfo("Same start, different end\n", id, seq, transcript, refSeq, similarity, seqLength, refSeqLength, 
                                    len_diff, upstreamSimilarity, u_similar_res, downstreamSimilarity, d_similar_res, stopCodonStateOneRound)
        elif user_transcript_start != ref_start and user_transcript_end == ref_end:
            frameshiftStateOneRound = "novel start"
        elif user_transcript_start != ref_start and user_transcript_end != ref_end:
            frameshiftStateOneRound = "novel transcript"        
        else:
            frameshiftStateOneRound = "todo1"
            errInfo("todo1\n", id, seq, transcript, refSeq, similarity, seqLength, refSeqLength, 
                                    len_diff, upstreamSimilarity, u_similar_res, downstreamSimilarity, d_similar_res, stopCodonStateOneRound)
   

    # update
        frameshiftStateFinal.append(frameshiftStateOneRound)
        frameshiftTranscriptFinal.append(transcript)
        stopCodonStateFinal.append(stopCodonStateOneRound)
        lengthChangeFinal.append(str(len_diff))
        startCodonStateFinal.append(startCodonStateOneRound)
    ############# write output
    #print(frameshiftStateFinal)
    #print(frameshiftTranscriptFinal)
    #print(stopCondanStateFinal)

    if (len(stopCodonStateFinal)==0):
        frameshiftStateFinal = ["no reference orf"]
       
    ofs.flush()
    
    ofs.write(gene+'\t')
    ofs.write(id+'\t')    
    ofs.write(str(user_transcript_start)+'\t')       
    ofs.write(str(user_transcript_end)+'\t')  
    ofs.write(classMap[min([frameMap[t] for t in frameshiftStateFinal])]+'\t')
    ofs.write(str( len(seq)+1)+'\t')    
    ofs.write(', '.join(frameshiftStateFinal)+'\t')
    ofs.write(', '.join(startCodonStateFinal)+'\t')
    ofs.write(', '.join(stopCodonStateFinal)+'\t')
    ofs.write(', '.join(frameshiftTranscriptFinal)+'\t')
    ofs.write(', '.join(lengthChangeFinal)+'\t')
    ofs.write('M'+seq+'\n')    
      

ofs.flush()    
ofs.close()
